# How to Access TESS Data: `astroquery`

## Introduction

By the end of this Notebook, you will:
- Understand how to query for TESS light curves for a single target
- Apply querying techniques to look for data from an example TESS Guest Investigator program
- Be able to recognize the appropriate syntax for data access in TIKE

To achieve the above goals, we'll perform some advanced queries with `astroquery.mast`. We'll also take a look at the two approaches to handling data, depending on whether you're working locally or on the TIKE platform.

## Imports and Setup
- The `Observations` module from `astroquery.mast` is needed to make the query and download the data
- `s3fs` lets us read data from the S3 bucket as though it were local

Note: If you are running this notebook on the TIKE, as recommended, you should not need to install or update your `astroquery` package. It "just works".

In [15]:
from astroquery.mast import Observations
from astropy.io import fits

import s3fs

Virtually all data from the TESS Mission are available for free on Amazon Web Services in public S3 bucket (as we'll see in the example below, S3 is missing data validation reports). There are two methods for access:
1. Read the files "locally" on TIKE (recommended)
2. Re-direct downloads from the MAST Servers to the AWS datacenter. [No AWS account required!](https://astroquery.readthedocs.io/en/latest/mast/mast.html#cloud-data-access).

Whichever method we chose, the querying method is identical. We'll start by activating cloud access, which is necessary for method 2.

In [2]:
# Enables downloads from the S3 bucket for method 2
Observations.enable_cloud_dataset()

INFO: Using the S3 STScI public dataset [astroquery.mast.cloud]


## Query for a specific target
In this example, we want to retrieve all TESS data (light curves, target pixel files, and data validation files) for the target TIC 7854182, a known $\delta$ Scuti star in an eclipsing binary (Chen et al. 2022, Kahraman Aliçavuş et al. 2017, Liakos & Niarchos 2017).

Feel free to use your favorite target here instead!

### A Slow Positional Query, for Completeness

We're going to perform a positional query to begin. This is the most generic type of MAST Search you can perform. The entered target name is converted to coordinates by Simbad and NED, then a query begins for any spatially overlapping Observations in the MAST Archive. As you might imagine, there is a limit on how quickly this type of search can be performed, even with an "exact match" search `radius=0`.

In [3]:
# Feel free to use your favorite target here instead
target_name = "TIC 7854182"

In [4]:
obs = Observations.query_object(target_name, radius="0s")

The results of our search are stored (as an Astropy Table) in the `obs` variable we set above. Let's print out the length of the table and then a random subset of the Observations returned.

In [5]:
# Print total number of Observations
print(f"TOTAL Observations available for {target_name}: \n{len(obs)}")

# Print every tenth row in the table
print(obs[::10])

TOTAL Observations available for TIC 7854182: 
47
intentType obs_collection provenance_name ... srcDen   obsid   distance
---------- -------------- --------------- ... ------ --------- --------
   science           TESS            SPOC ...    nan  27463640      0.0
   science           TESS            SPOC ...    nan  96861135      0.0
   science            PS1             3PI ... 5885.0   2578109      0.0
   science           HLSP           TASOC ...    nan 102001412      0.0
   science           HLSP            TICA ...    nan 100379453      0.0


Depending on when you run this query, you may see TESS, [PS1](https://archive.stsci.edu/panstarrs/), and [HLSP](archive.stsci.edu/hlsp). Evidently, such queries work very well for holistic searches, and there are a multitude of reasons you may want to know if you target has been observed by more than just TESS. It is also possible to filter the results from our generic search with indexing.

In [6]:
just_tess = obs[obs['obs_collection']=='TESS']
len(just_tess)

18

As of Jan 2024, this gives 18 results. Regardless of when you run this code, the number of results after filtering will be lower, since we've excluded other missions from this count.

However, as mentioned, this type of search is slow. It is particularly inefficient when we know the TICID, as there is a convenient shortcut we can take.

### A Convenient Shortcut
We can take advantage of a rather convenient shortcut, since we know two critical facts:
1. The data was collected by TESS (`obs_collection="TESS"`)
2. The TICID of our target (`target_name="####"`)

We'll use a new search function, the `Observations.query_criteria` search. The full list of queryable criteria is available on the [CAOM field descriptions page](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). Passing the criteria from our list to the search function will be sufficient to narrow down our observations, and has the advantage of being quicker. We are also certain to avoid any issues with the conversion of target name to coordinates.

**NOTE:** This shortcut only works because the TICID provides a consistent labeling framework for targets. Typically, this field is populated by the value submitted by the proposer in the APT file. That is to say: it is normally wildly inconsistent. Use with extreme caution, or potentially not at all, outside of the context of TESS.

In [7]:
obs = Observations.query_criteria(target_name="7854182", obs_collection="TESS")
obs[:5]

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str4,str10,str4,str4,str7,str7,str1,str47,float64,float64,str10,str14,int64,float64,float64,float64,float64,float64,str1,float64,str7,str1,int64,str47,str1,str79,str6,bool,float64,str8,str9
science,TESS,SPOC,Photometer,TESS,TESS,Optical,7854182,--,tess2020133194932-s0025-0000000007854182-0182-s,275.548647063955,47.568878067161,timeseries,"Ricker, George",3,58983.13551149,59008.80681946,120.0,600.0,1000.0,--,59050.0,G022062,--,25,CIRCLE ICRS 275.54864706 47.56887807 0.00138889,--,mast:TESS/product/tess2020133194932-s0025-0000000007854182-0182-s_lc.fits,PUBLIC,False,nan,27626482,70835966
science,TESS,SPOC,Photometer,TESS,TESS,Optical,7854182,--,tess2019199201929-s0014-s0026-0000000007854182,275.548647063955,47.568878067161,timeseries,"Ricker, George",3,58983.12717798611,59034.63613398148,120.0,600.0,1000.0,--,59092.0,G022062,--,26,CIRCLE 275.54864706 47.56887807 0.00138889,--,mast:TESS/product/tess2019199201929-s0014-s0026-0000000007854182-00353_dvt.fits,PUBLIC,False,nan,27740920,116847240
science,TESS,SPOC,Photometer,TESS,TESS,Optical,7854182,--,tess2020160202036-s0026-0000000007854182-0188-s,275.548647063955,47.568878067161,timeseries,"Ricker, George",3,59009.76794068287,59034.63613398148,120.0,600.0,1000.0,--,59064.0,G022062,--,26,CIRCLE 275.54864706 47.56887807 0.00138889,--,mast:TESS/product/tess2020160202036-s0026-0000000007854182-0188-s_lc.fits,PUBLIC,False,nan,27671902,116847248
science,TESS,SPOC,Photometer,TESS,TESS,Optical,7854182,--,tess2021175071901-s0040-0000000007854182-0211-s,275.548647063955,47.568878067161,timeseries,"Ricker, George",3,59390.15401666667,59418.35530668982,120.0,600.0,1000.0,--,59456.0,G04123,--,40,CIRCLE 275.54864706 47.56887807 0.00138889,--,mast:TESS/product/tess2021175071901-s0040-0000000007854182-0211-s_lc.fits,PUBLIC,False,nan,62346553,117015989
science,TESS,SPOC,Photometer,TESS,TESS,Optical,7854182,--,tess2021204101404-s0041-0000000007854182-0212-s,275.548647063955,47.568878067161,timeseries,"Ricker, George",3,59419.49001267361,59446.07981400463,120.0,600.0,1000.0,--,59477.0,G04123,--,41,CIRCLE 275.54864706 47.56887807 0.00138889,--,mast:TESS/product/tess2021204101404-s0041-0000000007854182-0212-s_lc.fits,PUBLIC,False,nan,62789558,120240735


Excellent. The next step is to go from `Observation` results to the underlying data files.

## Accessing the Underlying Data Files
We've been looking at `Observations`: an overaching colleciton of metadata and underlying data files. Now we'd like to get at the actual data, in this case a timeseries of our target star. The relevant astroquery function is `Observations.get_product_list()`. To keep this short, let's only pass the first `Observation` to this function. To get all of the light curves, just drop the `[0]` indexing.

In [8]:
prod = Observations.get_product_list(obs[0])
prod

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str8,str4,str10,str47,str33,str1,str81,str7,str28,str3,str1,str4,str20,str7,str63,int64,str8,str6,int64
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,full data validation report,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.pdf,INFO,--,DVR,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.pdf,5303784,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,full data validation report (xml),S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.xml,INFO,--,DVR,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.xml,67779,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Data validation mini report,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvm.pdf,INFO,Minimum Recommended Products,DVM,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvm.pdf,3245532,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,TCE summary report,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-01-00344_dvs.pdf,INFO,Minimum Recommended Products,DVS,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-01-00344_dvs.pdf,1144136,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Data validation time series,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvt.fits,SCIENCE,Minimum Recommended Products,DVT,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvt.fits,3729600,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Light curves,S,mast:TESS/product/tess2020133194932-s0025-0000000007854182-0182-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,b9bb72fe01,G022062,tess2020133194932-s0025-0000000007854182-0182-s_lc.fits,1877760,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Target pixel files,S,mast:TESS/product/tess2020133194932-s0025-0000000007854182-0182-s_tp.fits,SCIENCE,Minimum Recommended Products,TP,--,SPOC,spoc-4.0.36-20200520,G022062,tess2020133194932-s0025-0000000007854182-0182-s_tp.fits,45299520,27626482,PUBLIC,2


We got back quite a bit more than light curves. That's because there is actually a substantial amount of data associated with each TESS Observation. The `description` field is useful for using `Observations.filter_products()` for particular files. Full explanations of each of the product filters that can be used in the `filter_products` function are available on the [MAST API Product Field Description page](https://mast.stsci.edu/api/v0/_productsfields.html).

Here's a quick example of filtering for just the light curves:

In [9]:
# Sample filtering query
filt = Observations.filter_products(prod, description="Light curves")

Now, we're ready to access the data from the AWS S3 bucket!

### Method 1: TIKE
With TIKE, no download is necessary; it's as though you were accessing the files locally. There is a little bit of extra work  writing code, but it pays off in terms of access speeds.

In order to open files with TIKE, we'll need the cloud URI. Fortunately, there is a handle function to do just that: `Observations.get_cloud_uris`. We'll also need to enable `s3fs`, which lets Python access the S3 bucket as though it were local data.

In [10]:
# Get the cloud URI
c_uri = Observations.get_cloud_uris(filt)

# Enable Pythonic access of the S3 filesystem
fs = s3fs.S3FileSystem(anon=True)

Excellent. Now we just need one final `with` block to access the data in the file.

In [16]:
# Open the file in AWS: 'F' is the S3 file
with fs.open(c_uri[0], 'rb') as f:
     # Now actually read in the FITS file 
    with fits.open(f, 'readonly') as HDUlist:
        HDUlist.info()

Filename: <class 's3fs.core.S3File'>
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  LIGHTCURVE    1 BinTableHDU    161   18489R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        49   (11, 11)   int32   


### Method 2: Redirecting (local) Downloads to the Cloud

This is NOT the correct strategy on the TIKE Platform, since there's no need for a download.

When set to True, the `cloud_only` parameter in `download_products` skips any data products that are not available in the cloud; all TESS Mission data are available through AWS, so none of the selected data products should be skipped.

NOTE -- If you try to download the same file(s) more than once (e.g., by running the cell below multiple times), you should get the message "Found cached file" instead of "Downloading URL" in the printed manifest.

In [18]:
manifest = Observations.download_products(filt, cloud_only=True)

INFO: Found cached file ./mastDownload/TESS/tess2020133194932-s0025-0000000007854182-0182-s/tess2020133194932-s0025-0000000007854182-0182-s_lc.fits with expected size 1877760. [astroquery.mast.cloud]


All TESS light curves for TIC 7854182 have been downloaded, and we're ready to start our science!

## Bonus: Query for a specific Guest Investigator program
In this example, we want to retrieve all TESS data (light curves, target pixel files, and data validation files) associated with Guest Investigator program G05101 from Cycle 5 (PI: Susan Mullally).

Feel free to use any program here! The list of program IDs can be found at the [TESS GI List of Approved Programs](https://heasarc.gsfc.nasa.gov/docs/tess/approved-programs.html).

In [19]:
pid = "G05101"

In [20]:
obs_pid = Observations.query_criteria(obs_collection = "TESS",
                                  proposal_id = f"*{pid}*")

In [21]:
print(f"TOTAL Number of Observations available for {pid}: \n{len(obs_pid)}")

TOTAL Number of Observations available for G05101: 
49


In [22]:
display(obs_pid[:5])

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str4,str10,str4,str4,str7,str9,str1,str52,float64,float64,str10,str14,int64,float64,float64,float64,float64,float64,str1,float64,str34,str1,int64,str43,str1,str79,str6,bool,float64,str9,str9
science,TESS,SPOC,Photometer,TESS,TESS,Optical,383623835,--,tess2022330142927-s0059-0000000383623835-0248-s,270.628087535846,58.6272659486741,timeseries,"Ricker, George",3,59909.76229211806,59936.1858008912,120.0,600.0,1000.0,--,59962.0,G05101,--,59,CIRCLE 270.62808754 58.62726595 0.00138889,--,mast:TESS/product/tess2022330142927-s0059-0000000383623835-0248-s_lc.fits,PUBLIC,False,nan,113395560,210506880
science,TESS,SPOC,Photometer,TESS,TESS,Optical,5914520,--,tess2023096110322-s0064-0000000005914520-0257-s,193.312720118104,-18.5222248428656,timeseries,"Ricker, George",3,60040.63527681713,60067.43903234954,120.0,600.0,1000.0,--,60087.0,G05101,--,64,CIRCLE 193.31272012 -18.52222484 0.00138889,--,mast:TESS/product/tess2023096110322-s0064-0000000005914520-0257-s_lc.fits,PUBLIC,False,nan,139084486,254133156
science,TESS,SPOC,Photometer,TESS,TESS,Optical,298411553,--,tess2022330142927-s0059-0000000298411553-0248-s,149.476234232293,85.4946883170451,timeseries,"Ricker, George",3,59909.7650365625,59936.18904974537,120.0,600.0,1000.0,--,59962.0,G05101_G05084_G05115_G05081,--,59,CIRCLE 149.47623423 85.49468832 0.00138889,--,mast:TESS/product/tess2022330142927-s0059-0000000298411553-0248-s_lc.fits,PUBLIC,False,nan,113392111,210500653
science,TESS,SPOC,Photometer,TESS,TESS,Optical,219097989,--,tess2022357055054-s0060-0000000219097989-0249-s,269.658310781969,66.7811423559296,timeseries,"Ricker, George",3,59936.40188498842,59962.08243238426,120.0,600.0,1000.0,--,59982.0,G05101,--,60,CIRCLE 269.65831078 66.78114236 0.00138889,--,mast:TESS/product/tess2022357055054-s0060-0000000219097989-0249-s_lc.fits,PUBLIC,False,nan,114309714,211965772
science,TESS,SPOC,Photometer,TESS,TESS,Optical,420814525,--,tess2022244194134-s0056-0000000420814525-0243-s,330.794887332661,18.8843189579296,timeseries,"Ricker, George",3,59824.761134409724,59852.642480821756,120.0,600.0,1000.0,--,59873.0,G05101_G05015_G05144_G05069,--,56,CIRCLE 330.79488733 18.88431896 0.00138889,--,mast:TESS/product/tess2022244194134-s0056-0000000420814525-0243-s_lc.fits,PUBLIC,False,nan,97942919,184143980


Next, we will retrieve the list of data products that are associated with each Observation.


In [23]:
# Pick which products we want to retrieve
data_prod_pid = Observations.get_product_list(obs_pid)

In [24]:
print(f"Number of TESS data products available for {pid}: \n{len(data_prod_pid)}")

Number of TESS data products available for G05101: 
106


In [27]:
display(prod)

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str8,str4,str10,str47,str33,str1,str81,str7,str28,str3,str1,str4,str20,str7,str63,int64,str8,str6,int64
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,full data validation report,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.pdf,INFO,--,DVR,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.pdf,5303784,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,full data validation report (xml),S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.xml,INFO,--,DVR,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvr.xml,67779,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Data validation mini report,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvm.pdf,INFO,Minimum Recommended Products,DVM,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvm.pdf,3245532,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,TCE summary report,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-01-00344_dvs.pdf,INFO,Minimum Recommended Products,DVS,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-01-00344_dvs.pdf,1144136,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Data validation time series,S,mast:TESS/product/tess2020135030118-s0025-s0025-0000000007854182-00344_dvt.fits,SCIENCE,Minimum Recommended Products,DVT,--,SPOC,b9bb72fe01,G022062,tess2020135030118-s0025-s0025-0000000007854182-00344_dvt.fits,3729600,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Light curves,S,mast:TESS/product/tess2020133194932-s0025-0000000007854182-0182-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,b9bb72fe01,G022062,tess2020133194932-s0025-0000000007854182-0182-s_lc.fits,1877760,27626482,PUBLIC,3
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Target pixel files,S,mast:TESS/product/tess2020133194932-s0025-0000000007854182-0182-s_tp.fits,SCIENCE,Minimum Recommended Products,TP,--,SPOC,spoc-4.0.36-20200520,G022062,tess2020133194932-s0025-0000000007854182-0182-s_tp.fits,45299520,27626482,PUBLIC,2


As above, if we decide that we only want to download, e.g., the light curve files, we can further filter the products list based on the columns/fields in the table.

For our purposes, this step is optional; I'm going to download **all** of the available data products for this program ID.

In [ ]:
# OPTIONAL
# Select which files to download from the S3 bucket by applying additional filters
# filt_prod = Observations.filter_products(data_prod_pid, description='Light curves')

Now, we're ready to download the data from the AWS S3 bucket!

By default, all downloads will be placed in a `./mastDownload/` directory on the TIKE. If you'd like to change this directory, use the `download_dir` parameter in the `download_products` functions. For example, you may want to place all products downloaded for GI program G05101 in a directory named `./G05101`, as shown below.


In [29]:
manifest = Observations.download_products(data_prod_pid[:10], download_dir=f'{pid}', cloud_only=True)

ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvr.xml with expected size 374231. [astroquery.mast.cloud]


ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-01-00810_dvs.pdf with expected size 703837. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvt.fits with expected size 53426880. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000207440438-0243-s/tess2022244194134-s0056-0000000207440438-0243-s_lc.fits with expected size 2039040. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000207440438-0243-s/tess2022244194134-s0056-0000000207440438-0243-s_tp.fits with expected size 49193280. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000219089401-0243-s/tess2022244194134-s0056-0000000219089401-0243-s_lc.fits with expected 

If you scroll through the above manifest, you may notice an error:
```
ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]
WARNING: Skipping file... [astroquery.mast.observations]
```

So, what happened here? Let's check the manifest to see which files were not downloaded.


In [30]:
display(manifest[manifest['Status']!='COMPLETE'])

Local Path,Status,Message,URL
str135,str8,object,object
G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvr.pdf,SKIPPED,None,None
G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvm.pdf,SKIPPED,None,None


When we say that "ALL" data from the TESS Mission are available in AWS S3 buckets, there is a small caveat.

The TESS Full Data Validation Reports (`*_dvr.pdf`) and Mini Data Validation Reports (`*_drm.pdf`), which are produced for all TCEs associated with a particular host star, are **not currently** hosted on AWS.

As MAST moves towards a more cloud-based model for data access, MAST is considering adding these types of data products to the AWS S3 bucket, but for now, if we want to download them to our local directory, we'll need to download them directly from MAST. To do this, we set `cloud_only=False`. If set to False and cloud data access is enabled (`enable_cloud_dataset` above), files that are not found in the cloud will be downloaded from MAST.

In [31]:
manifest = Observations.download_products(data_prod_pid[:10], download_dir=f'{pid}', cloud_only=False)

ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


 [Done]
INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvr.xml with expected size 374231. [astroquery.mast.cloud]

ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


 [Done]
INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-01-00810_dvs.pdf with expected size 703837. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvt.fits with expected size 53426880. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000207440438-0243-s/tess2022244194134-s0056-0000000207440438-0243-s_lc.fits with expected size 2039040. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000207440438-0243-s/tess2022244194134-s0056-0000000207440438-0243-s_tp.fits with expected size 49193280. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000219089401-0243-s/tess2022244194134-s0056-0000000219089401-0243-s_lc.fits with e

All TESS data products (light curves, target pixel files, and data validation files) associated with Guest Investigator program G05101 have been downloaded

## About this Notebook

**Author:** Hannah M. Lewis, STScI Data Scientist. Edits by Nicole Schanche and Thomas Dutkiewicz.

**Last updated:** 5 Jan 2023

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/>